# Schematic

1. Load in SA1 data. Denote this as SA1
2. Split into a matrix for each party. Denote this as SA1_partyab
2. Shrink to NSW
3. Take the data and split it into two value:
    - A normalised by year matrix so that each SA1 now measures the percentage contribution to the state voteshare. Denote this as M_NSW_partyab
    - A vector of the total voteshare which overall. X_NSW_partyab
4. Perform VAR on the M_NSW_parytyab to predict the next election. Use to predict M_NSW_partyab at t+1. Denote this as predper_NSW_partyab.
5.  Perform linear regression on X_NSW_paryab to predict the next total voteshare with corresponding uncertainty at t+1. Weight polling uncertainty and trend uncertainty according to Bayesian average. Denote this as predtot_NSW_partyab.
6. Multiply these two values together to form a prediction of SA1_partyab at t+1.

In [2]:
import os,sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.api import VAR

elections = ['2004','2007','2010','2013','2016','2019','2022']
files = ['pollingplaces', 'primaries', 'tpptcp']
def makeid(row):
    pollingid = str(row["PollingPlaceID"])
    partyid = row["PartyAb"]

    if type(partyid) != str:
        if np.isnan(partyid):
            partyid = row["Surname"].upper()
    elif partyid.upper() == "IND":
        partyid = row["Surname"].upper()

    return str(row["PollingPlaceID"])+str(partyid)

dfs = []
for electionyear in elections:
    for file in os.listdir(f"data/{electionyear}/{files[1]}"):
        file = os.path.join(f"data/{electionyear}/{files[1]}",file)
        if 'prdelms' in file:
            ppid_locations = pd.read_csv(file)
            continue
        boothresults = pd.read_csv(file,header=1)
        try:
            boothresults['id'] = boothresults.apply(makeid, axis=1)
        except KeyError as e:
            print(file)
            raise e     
        boothresults['year'] = electionyear
        boothresults['uniqueid'] = boothresults.apply(lambda row: str(row["id"])+str(row["year"]), axis=1)

        boothresults.set_index('uniqueid', inplace=True)
        dfs.append(boothresults)

df = pd.concat(dfs)
df = df[df['StateAb']=='NSW']
# df = df[df['year']==2022]

df.head()

,StateAb,DivisionID,DivisionNm,PollingPlaceID,PollingPlace,CandidateID,Surname,GivenNm,BallotPosition,SittingMemberFl,PartyAb,PartyNm,OrdinaryVotes,Swing,id,year,Elected,HistoricElected
uniqueid,,,,,,,,,,,,,,,,,,
1HAN2004,NSW,103,Banks,1,Beverly Hills North,15349,STAVRINOS,Harry,1,NaN,HAN,Pauline Hanson's One Nation (NSW Division),49,-2.52,1HAN,2004,NaN,NaN
1GRN2004,NSW,103,Banks,1,Beverly Hills North,14170,MAKIN,Stephen,2,NaN,GRN,The Greens,107,1.58,1GRN,2004,NaN,NaN
1LP2004,NSW,103,Banks,1,Beverly Hills North,14709,GRAY,Roger,3,NaN,LP,Liberal,990,6.33,1LP,2004,NaN,NaN
1FFP2004,NSW,103,Banks,1,Beverly Hills North,15410,BRISCOE-HOUGH,Greg,4,NaN,FFP,Family First,20,0.79,1FFP,2004,NaN,NaN
1DEM2004,NSW,103,Banks,1,Beverly Hills North,15556,CLYBURN,Mark,5,NaN,DEM,Australian Democrats,15,-1.99,1DEM,2004,NaN,NaN


In [10]:
pivot_df = df.pivot(index="id",columns='year', values='OrdinaryVotes')
covmat = pivot_df.fillna(0).T.corr()

year
2004    1181252
2007    1462283
2010    1296546
2013    1232977
2016    1400665
2019    1363996
2022    1290950
dtype: int64

In [32]:
sa1_alp = pivot_df[pivot_df.index.str.contains("ALP")].fillna(0).astype(int).T
X_NSW_alp = sa1_alp.sum(axis=0)
M_NSW_alp = sa1_alp /X_NSW_alp

In [33]:
model = VAR(M_NSW_alp)

C:\Users\pcah8798\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency 3AS-JAN will be used.
  self._init_dates(dates, freq)


In [36]:
model.fit(3)

ValueError: x contains one or more constant columns. Column(s) 8917 are constant. Adding a constant with trend='c' is not allowed.